In [63]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
from sklearn.model_selection import train_test_split
from regression_dsets_large_names import regression_dsets_large_names
import numpy as np
import matplotlib.pyplot as plt
import pmlb
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
import data
import fit
from process_results import process_results, aggregate_results

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load results from a directory

In [ ]:
!python process_results.py

In [69]:
# these all have ols, ridge, sta, lasso with appropriate hyperparams
# beta=gaussian_noise=1e-1
# beta=gaussian_noise=0 
# beta=onehot_noise=0  
# beta=onehot_noise=1e-1 

save_dir = 'results'
# folders = ['beta=gaussian_noise=1e-1', 'beta=gaussian_noise=0',
#            'beta=onehot_noise=0', 'beta=onehot_noise=1e-1']
folders = os.listdir('/scratch/users/vision/yu_dl/raaz.rsk/double_descent/all_linear')

# linear plots

In [ ]:
for folder in folders:
    out_dir = oj('/scratch/users/vision/yu_dl/raaz.rsk/double_descent/all_linear', folder)
    df = pd.read_pickle(oj(out_dir, 'processed.pkl'))
    
    R, C = 2, 2
    SIZE = 8
    plt.figure(figsize=(C * SIZE, R * SIZE))
    plt.subplot(R, C, 1)
    plt.axhline(df.mse_zero.values[0], lw=4, color='gray', alpha=0.4, label='trivial')
    for name, curve in tqdm(df.iterrows()):
    #     print(name, curve)
        model_type = name[3]
        reg_param = name[4]
        l = str(model_type) + ' ' + str(reg_param)
        if model_type == 'ols':
            ls = '--'
        elif model_type == 'ridge':
            ls = ':'
        elif model_type == 'linear_sta':
            ls = '-.'
        elif model_type == 'lasso':
            ls = '-'


        kwargs = {'label': l, 'alpha': 0.5, 'lw': 2, 'linestyle': ls}

        plt.subplot(R, C, 1)
    #     plt.plot(curve.ratio, curve.mse_test, 
        plt.plot(curve.ratio, np.square(curve.bias) + curve['var'], **kwargs)
        plt.xlabel('p / n')
        plt.ylabel('test mse')


        plt.subplot(R, C, 2)
        plt.plot(curve.ratio, curve.mse_train, **kwargs)
        plt.xlabel('p / n')
        plt.ylabel('train mse')
        
        
        plt.subplot(R, C, 3)
        plt.plot(curve.ratio, np.square(curve.bias), **kwargs)
        plt.xlabel('p / n')
        plt.ylabel('bias')
        
        plt.subplot(R, C, 4)
        plt.plot(curve.ratio, curve['var'], **kwargs)
        plt.xlabel('p / n')
        plt.ylabel('var')
        
        # wnorm also recorded

    for i in range(1, R * C + 1):
        plt.subplot(R, C, i)
        plt.xscale('log')
        plt.yscale('log')

    #     s = f'{model_type}_{dset}_{dset_name}+p={num_features}+lambda={reg_param}+noise_mult={noise_mult}'
    plt.subplot(R, C, 1)
    plt.legend()
    s = folder
    plt.suptitle(s)
    plt.tight_layout()
    # plt.show()
    plt.savefig(oj(save_dir, s + '.pdf'))
#     plt.show()
    plt.close()